<a href="https://colab.research.google.com/github/ainnurnazri/Flowise/blob/main/UJAR_Abstrak_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving UMUM_AKADEMIKA_74.pdf to UMUM_AKADEMIKA_74.pdf


In [ ]:
from google.colab import files
import fitz  # PyMuPDF
import re

pdf_path = 'UMUM_AKADEMIKA_74.pdf'
selected_pages = [0]  # Page 4, 7, 8 (0-based indexing)

no_break_patterns = [
    r'S\.W\.T', r'Prof\.', r'Dr\.', r'et al\.', r'Bil\.', r'et. al\.', r'et.al\.', r'r.a\.',
    r'pp\.', r'vol\.', r'no\.', r'fig\.', r'ed\.'
]
no_break_re = re.compile('|'.join(no_break_patterns), re.IGNORECASE)

bullet_points = [
    '•', '▪', '‣', '⁃', '○', '●', '■', '□', '◆', '▶', '➢', '⦿',
    '◯', '⭘', '〇', '◦', '⦾', '⦿', '',
    '- ', '* ', '+ '
]
bullet_re = re.compile('|'.join([re.escape(bp) for bp in bullet_points]))

remove_patterns = [
    r'^\d+$',
    r'^Jadual\s+\d+.*$',
    r'^\s*Bil\s*[\|\s].*$',
    r'^\|.*\|$',
    r'^\d+(?:\.\d+){1,10}\s+.*$',
    r'^(?i:M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3}))$'
]
remove_re = re.compile('|'.join(remove_patterns))
number_only_re = re.compile(r'^\s*\d+\s*$')

doc = fitz.open(pdf_path)
extracted_text = ""

for page_num in selected_pages:
    page = doc.load_page(page_num)
    page_text = page.get_text("text", flags=fitz.TEXT_PRESERVE_LIGATURES | fitz.TEXT_PRESERVE_WHITESPACE)
    lines = [l.strip() for l in page_text.splitlines()]

    if page_num == 3:
        # Get title from page 4 until first blank line
        title_lines = []
        for line in lines:
            if not line:
                break
            title_lines.append(line)
        title_text = " ".join(title_lines).strip()
        extracted_text += title_text + "\n\n"
        continue

    # Clean other pages
    clean_lines = []
    for raw in lines:
        line = raw.strip()
        if not line or remove_re.fullmatch(line):
            continue
        line = bullet_re.sub(' ', line)
        line = re.sub(r'\s+', ' ', line).strip()
        clean_lines.append(line)

    page_content = " ".join(clean_lines)

    # Restore abbreviations
    placeholder_map = {}
    for idx, pat in enumerate(no_break_patterns):
        matches = re.findall(pat, page_content, flags=re.IGNORECASE)
        for match in matches:
            placeholder = f"__ABBR_{idx}_{len(placeholder_map)}__"
            placeholder_map[placeholder] = match
            page_content = page_content.replace(match, placeholder)

    # Split into sentences for better formatting
    sentences = re.split(r'(?<=[.])\s+(?=[A-Z0-9])', page_content)
    restored_sentences = []
    for s in sentences:
        for placeholder, original in placeholder_map.items():
            s = s.replace(placeholder, original)
        restored_sentences.append(s.strip())

    # Add line breaks for ABSTRAK / ABSTRACT
    text_block = "\n".join(restored_sentences)
    text_block = re.sub(r'\b(ABSTRAK|ABSTRACT)\b', r'\n\1\n', text_block)

    extracted_text += text_block.strip() + "\n\n"

doc.close()

# Final cleanup
extracted_text = re.sub(r'\n{3,}', '\n\n', extracted_text.strip())

# Save output
with open('UMUM_AKADEMIKA_74_abstrak.txt', 'w', encoding='utf-8') as f:
    f.write(extracted_text)

files.download('UMUM_AKADEMIKA_74_abstrak.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>